This notebook calculates the change in indirect damage for specified hazard maps

In [1]:
# Imports
import configparser
from pathlib import Path
import pathlib
from direct_damages import damagescanner_rail_track as ds
import pandas as pd
import geopandas as gpd
# import datetime
from ci_adapt_utilities import *
import pickle
import networkx as nx

In [2]:
# Load configuration with ini file (created running config.py)
config_file=r'C:\repos\ci_adapt\config_ci_adapt.ini'
config = configparser.ConfigParser()
config.read(config_file)

p = Path('..')
hazard_type = config.get('DEFAULT', 'hazard_type')
infra_type = config.get('DEFAULT', 'infra_type')
country_code = config.get('DEFAULT', 'country_code')
country_name = config.get('DEFAULT', 'country_name')
hazard_data_subfolders = config.get('DEFAULT', 'hazard_data_subfolders')
asset_data = config.get('DEFAULT', 'asset_data')
vulnerability_data = config.get('DEFAULT', 'vulnerability_data')
data_path = Path(pathlib.Path.home().parts[0]) / 'Data'
interim_data_path = data_path / 'interim' / 'collected_flood_runs'

In [3]:
# Define costs for different transport modes
average_train_load_tons = (896+1344+2160+1344+896+896+1344+1512+896+390)/10 # in Tons per train. Source: Kennisinstituut voor Mobiliteitsbeleid. 2023. Cost Figures for Freight Transport – final report
average_train_cost_per_ton_km = (0.014+0.018+0.047+0.045)/4 # in Euros per ton per km. Source: Kennisinstituut voor Mobiliteitsbeleid. 2023. Cost Figures for Freight Transport – final report
average_road_cost_per_ton_km = (0.395+0.375+0.246+0.203+0.138+0.153+0.125+0.103+0.122+0.099)/10 # in Euros per ton per km. Source: Kennisinstituut voor Mobiliteitsbeleid. 2023. Cost Figures for Freight Transport – final report

In [4]:
def calculate_new_paths(graph0, shortest_paths, disrupted_edges):
    graph=graph0.copy()
    for u,v in set(disrupted_edges):
        graph.remove_edge(u,v,0)
        
    disrupted_shortest_paths={}
    for (origin,destination), (nodes_in_spath,demand) in shortest_paths.items():
        edges_in_spath=[(nodes_in_spath[i],nodes_in_spath[i+1]) for i in range(len(nodes_in_spath)-1)]
        if set(disrupted_edges).isdisjoint(edges_in_spath):
            continue
        else:
            try:
                disrupted_shortest_paths[(origin,destination)] = (nx.shortest_path(graph, origin, destination, weight='weight'), demand)
            except nx.NetworkXNoPath:
                print(f'No path between {origin} and {destination}. Cannot ship by train.')
                disrupted_shortest_paths[(origin,destination)] = (None, demand)
                continue
    
    return disrupted_shortest_paths

def calculate_economic_impact_shortest_paths(graph, shortest_paths, disrupted_shortest_paths, average_train_load_tons, average_train_cost_per_ton_km, average_road_cost_per_ton_km):
    # Initialize the economic impact variable
    economic_impact = 0
    # Loop through the edges where there is a change in flow
    for (origin, destination), (nodes_in_path, demand) in disrupted_shortest_paths.items():
        length_old_path=0
        for i in range(len(shortest_paths[(origin, destination)][0])-1):
            length_old_path += graph.edges[shortest_paths[(origin, destination)][0][i], shortest_paths[(origin, destination)][0][i+1], 0]['length']/1000
                     
        if (nodes_in_path is None) or ('_d' in str(nodes_in_path)):
            economic_impact += demand*average_train_load_tons*(average_road_cost_per_ton_km-average_train_cost_per_ton_km)*length_old_path
            continue


        else:
            length_new_path=0
            for i in range(len(nodes_in_path)-1):
                length_new_path += graph.edges[nodes_in_path[i], nodes_in_path[i+1], 0]['length']/1000
            economic_impact += demand*average_train_load_tons*average_train_cost_per_ton_km*(length_new_path-length_old_path)
        

    return economic_impact


def _inspect_graph(graph):
    edge_capacities_types = []
    edge_weights_types = []
    node_demands_types = []

    for _, _, attr in graph.edges(data=True):
        if 'capacity' in attr:
            edge_capacities_types.append(type(attr['capacity']))
        if 'weight' in attr:
            edge_weights_types.append(type(attr['weight']))

    for _, attr in graph.nodes(data=True):
        if 'demand' in attr:
            node_demands_types.append(type(attr['demand']))

    return edge_capacities_types, edge_weights_types, node_demands_types
    
def create_virtual_graph(graph):
    max_weight_graph = max(attr['weight'] for _, _, attr in G.edges(data=True))
    print('Max weight: '+str(max_weight_graph))
    max_capacity_graph = max(attr['capacity'] for _, _, attr in G.edges(data=True))
    print('Max capacity: '+str(max_capacity_graph))

    # create a virtual node with dummy nodes
    graph_v=graph.copy()
    # convert to int
    for u, v, key, attr in G.edges(keys=True, data=True):
        graph_v.add_edge((str(u) + '_d'), (str(v) + '_d'), **attr)

    for u in G.nodes:
        graph_v.add_edge(u,(str(u) + '_d'),capacity=max_capacity_graph*100,weight=int(round(1e10,0)))
        graph_v.add_edge((str(u) + '_d'),u,capacity=max_capacity_graph*100,weight=0)

    # verify capacities, weights and demands are integers
    edge_capacities_types, edge_weights_types, node_demands_types = _inspect_graph(graph_v)

    if {type(int())} == set(list(edge_capacities_types) + list(edge_weights_types) + list(node_demands_types)):
        print('Success: only int type values')
    else: 
        print('Warning! Not all values are integers')

    return graph_v

def recalculate_disrupted_edges(G, assets, disrupted_edges, fully_protected_assets, unexposed_osm_ids):
    # list of osm_ids of adapted assets
    adapted_osm_ids=assets.loc[assets.index.isin(fully_protected_assets)]['osm_id'].values
    available_osm_ids = np.unique(np.concatenate((unexposed_osm_ids, adapted_osm_ids)))
    available_edges=[]
    for (u,v) in disrupted_edges:
        # get the attributes of the edge
        osm_ids_edge = G.edges[(u,v,0)]['osm_id'].split(';')
        osm_ids_edge = [ids.strip() for ids in osm_ids_edge]

        # check if all the osm_ids of the edge are in the list of adapted assets
        if set(osm_ids_edge).issubset(available_osm_ids):
            available_edges.append((u,v))
        
    adapted_disrupted_edges = [edge for edge in disrupted_edges if edge not in available_edges]

    return adapted_disrupted_edges

In [5]:
def preprocess_assets(assets_path):
    assets = gpd.read_file(assets_path)
    assets = gpd.GeoDataFrame(assets).set_crs(4326).to_crs(3857)
    assets = assets.loc[assets.geometry.geom_type == 'LineString']
    assets = assets.rename(columns={'railway' : 'asset'})
    assets = assets[assets['railway:traffic_mode']!='"passenger"']
    assets = assets[assets['asset']=='rail']
    assets = assets.reset_index(drop=True)
    
    return assets

def filter_adapted_assets(assets, adaptation_area):
    filtered_assets = gpd.overlay(assets, adaptation_area, how='intersection')
    adapted_assets = assets.loc[(assets['osm_id'].isin(filtered_assets['osm_id']))]

    return adapted_assets


In [6]:
# Read exposure data (OSM, OpenStreetMap contributors (2024) / osm-flex)
assets_path = data_path / asset_data
assets=preprocess_assets(assets_path)

# Add buffer to assets to do area intersect and create dictionaries for quicker lookup
buffered_assets = ds.buffer_assets(assets)
geom_dict = assets['geometry'].to_dict()
type_dict = assets['asset'].to_dict()

print(f"{len(assets)} assets loaded.")

# Read vulnerability and maximum damage data from Nirandjan, S., et al. (2024)
curve_types = {'rail': ['F8.1']}
infra_curves, maxdams = ds.read_vul_maxdam(data_path, hazard_type, infra_type)
max_damage_tables = pd.read_excel(data_path / vulnerability_data / 'Table_D3_Costs_V1.0.0.xlsx',sheet_name='Cost_Database',index_col=[0])
print(f'Found matching infrastructure curves for: {infra_type}')

2822 assets loaded.
Found matching infrastructure curves for: rail


In [7]:
#load pickled shortest paths, disrupted edges, shortest paths, graph
shortest_paths = pickle.load(open(data_path / 'interim' / 'indirect_damages' / 'shortest_paths.pkl', 'rb'))
disrupted_edges_by_basin = pickle.load(open(data_path / 'interim' / 'indirect_damages' / 'disrupted_edges_by_basin.pkl', 'rb'))
G = pickle.load(open(data_path / 'interim' / 'indirect_damages' / 'G.pkl', 'rb'))
disrupted_shortest_paths = pickle.load(open(data_path / 'interim' / 'indirect_damages' / 'disrupted_shortest_paths.pkl', 'rb'))
event_impacts = pickle.load(open(data_path / 'interim' / 'indirect_damages' / 'event_impacts.pkl', 'rb'))
print('Loaded data from baseline impact assessment')

Loaded data from baseline impact assessment


In [8]:
G_v0=create_virtual_graph(G)
G_v=G_v0.copy()

Max weight: 30076136
Max capacity: 1
Success: only int type values


In [9]:
#Define an area to adapt
#adapt_path = Path(r'C:\Data\Floods\basins\hybas_eu_2080411370.shp')
adapt_path=Path(r'C:\Data\interim\adaptations\test_haz_level_adapt.geojson')

# Load the polygon of the protected area
protected_area = gpd.read_file(adapt_path).to_crs(3857)

# Filter assets that are within the protected area
adapted_assets=filter_adapted_assets(assets, protected_area)


In [10]:
# Define hazard-asset overlays to calculate adaptation for:
return_periods=['H','M','L']
basin_list=[2080418880, 2080416210, 2080416200, 2080411370]

In [11]:
rp=return_periods[1]
bas=basin_list[-1]

In [12]:
# open pickled hazard-asset overlay and hazard intensity data
with open(interim_data_path / f'overlay_assets_flood_DERP_RW_{rp}_4326_{bas}.pkl', 'rb') as f:
    overlay_assets = pickle.load(f)
with open(interim_data_path / f'hazard_numpified_flood_DERP_RW_{rp}_4326_{bas}.pkl', 'rb') as f:
    hazard_numpified_list = pickle.load(f)

# optionally to calculate the maximum intensity for each hazard point, this can be used, else a float can be used
max_intensity = []
for asset_id in adapted_assets.index:
    max_intensity.append(retrieve_max_intensity_by_asset(asset_id, overlay_assets, hazard_numpified_list))


In [19]:
# add new columns fragility_mod and haz_mod
adapted_assets['fragility_mod'] = 1 #[0.3, 0.5, 0.8] #fraction [example considering no reduction] (1 = no reduction, 0 = invulnerable asset) DUMMY DATA FOR TESTING
# adapted_assets['haz_mod'] = [np.max(x) for x in max_intensity] #meters [example adding wall of maximum flooding depth + 1 meter] (0 = no reduction in hazard intensity, 0.5 = 0.5 meter reduction in hazard intensity) DUMMY DATA FOR TESTING consider raising railway 0.5 meters
adapted_assets['haz_mod'] = [np.max(x) if len(x) > 0 else 0 for x in max_intensity]

# TODO: automate infrastructure curve deduction from dictionary keys, now running with curve F8.1
hazard_intensity = infra_curves['F8.1'].index.values
fragility_values = (np.nan_to_num(infra_curves['F8.1'].values,nan=(np.nanmax(infra_curves['F8.1'].values)))).flatten()
maxdams_filt=max_damage_tables[max_damage_tables['ID number']=='F8.1']['Amount']

adaptation_run = run_damage_reduction_by_asset(geom_dict, overlay_assets, hazard_numpified_list, adapted_assets, hazard_intensity, fragility_values, maxdams_filt, reporting=False)

#TODO Check with economist: ammortization of adaptation cost over years of adaptation scenario
    #NEXT: adaptation_run returns (collect_inb_bl, collect_inb_adapt, adaptation_cost). These can be used to calculate the EAD for the adapted scenario (and damage reduction), and compare with the adaptation cost, which must be ammortized over the years of the adaptation scenario.


c:\Users\peregrin\AppData\Local\anaconda3\envs\ci_adapt_env\Lib\site-packages\geopandas\geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


2024-07-30 17:10:15 - Calculating adapted damages for assets...


100%|██████████| 43/43 [00:46<00:00,  1.08s/it]

11 assets with no change.


In [14]:
# For a given hazard map overlay, find all the assets that are fully protected
fully_protected_assets=[asset_id for asset_id, damages in adaptation_run[1].items() if damages[0]==0 and damages[1]==0]

# For a given hazard map overlay, find all assets that are not exposed to flooding
unexposed_assets=[asset_id for asset_id in assets.index if asset_id not in overlay_assets.asset.values]
unexposed_osm_ids=assets.loc[assets.index.isin(unexposed_assets)]['osm_id'].values

# find the disrupted edges and paths under adapted conditions
disrupted_edges_by_basin_adapted = {}
disrupted_shortest_paths_adapted = {}
event_impacts_adapted = {}

for hazard_map in disrupted_edges_by_basin.keys():
    print(f'\n-- CALCULATIONS FOR {hazard_map} --')
    # find edges that will no longer be disrupted
    disrupted_edges = disrupted_edges_by_basin[hazard_map]
    print('disrupted_edges baseline: ', disrupted_edges)
    disrupted_edges_by_basin_adapted[hazard_map] = recalculate_disrupted_edges(G, assets, disrupted_edges, fully_protected_assets, unexposed_osm_ids)
    print('disrupted_edges_adapted: ', disrupted_edges_by_basin_adapted[hazard_map])

    disrupted_shortest_paths_adapted[hazard_map]=calculate_new_paths(G_v0, shortest_paths, disrupted_edges_by_basin_adapted[hazard_map])

    if disrupted_shortest_paths_adapted[hazard_map] == {}: # No disrupted paths, no economic impact
        print(f'No shortest paths disrupted for {hazard_map}. No economic impact.')
        event_impacts_adapted[hazard_map]=0
        continue

    impact=calculate_economic_impact_shortest_paths(G_v, shortest_paths, disrupted_shortest_paths_adapted[hazard_map], average_train_load_tons, average_train_cost_per_ton_km, average_road_cost_per_ton_km)
    event_impacts_adapted[hazard_map]=impact
    print(hazard_map, impact)
    




-- CALCULATIONS FOR flood_DERP_RW_H_4326_2080411370 --
disrupted_edges baseline:  [(397, 405), (398, 2349), (405, 397), (460, 2452), (461, 463), (463, 461), (1437, 2650), (1445, 1449), (1449, 1445), (1454, 2452), (1457, 1468), (1466, 1468), (1468, 1457), (1468, 1466), (1479, 1480), (1480, 1479), (2349, 398), (2452, 460), (2452, 1454), (2650, 1437)]
disrupted_edges_adapted:  []
No shortest paths disrupted for flood_DERP_RW_H_4326_2080411370. No economic impact.

-- CALCULATIONS FOR flood_DERP_RW_H_4326_2080416210 --
disrupted_edges baseline:  [(166, 2916), (203, 1668), (461, 463), (462, 532), (463, 461), (532, 462), (598, 2727), (717, 1737), (1392, 1940), (1394, 2728), (1398, 1937), (1433, 1438), (1437, 2650), (1438, 1433), (1654, 1669), (1668, 203), (1669, 1654), (1733, 1738), (1734, 1736), (1735, 1738), (1736, 1734), (1736, 1737), (1737, 1736), (1737, 717), (1738, 1735), (1738, 1733), (1937, 1398), (1940, 1392), (2650, 1437), (2727, 598), (2728, 1394), (2871, 2913), (2913, 2916), (29

In [15]:

for hazard_map in disrupted_edges_by_basin.keys():
    print(f'\n-- CALCULATIONS FOR {hazard_map} --')
    # find edges that will no longer be disrupted
    disrupted_edges = disrupted_edges_by_basin[hazard_map]
    print('disrupted_edges baseline: ', disrupted_edges)
    disrupted_edges_by_basin_adapted[hazard_map] = recalculate_disrupted_edges(G, assets, disrupted_edges, fully_protected_assets, unexposed_osm_ids)
    print('disrupted_edges_adapted: ', disrupted_edges_by_basin_adapted[hazard_map])

    disrupted_shortest_paths_adapted[hazard_map]=calculate_new_paths(G_v0, shortest_paths, disrupted_edges_by_basin_adapted[hazard_map])

    if disrupted_shortest_paths_adapted[hazard_map] == {}: # No disrupted paths, no economic impact
        print(f'No shortest paths disrupted for {hazard_map}. No economic impact.')
        event_impacts_adapted[hazard_map]=0
        continue

    impact=calculate_economic_impact_shortest_paths(G_v, shortest_paths, disrupted_shortest_paths_adapted[hazard_map], average_train_load_tons, average_train_cost_per_ton_km, average_road_cost_per_ton_km)
    event_impacts_adapted[hazard_map]=impact
    print(hazard_map, impact)
    




-- CALCULATIONS FOR flood_DERP_RW_H_4326_2080411370 --
disrupted_edges baseline:  [(397, 405), (398, 2349), (405, 397), (460, 2452), (461, 463), (463, 461), (1437, 2650), (1445, 1449), (1449, 1445), (1454, 2452), (1457, 1468), (1466, 1468), (1468, 1457), (1468, 1466), (1479, 1480), (1480, 1479), (2349, 398), (2452, 460), (2452, 1454), (2650, 1437)]
disrupted_edges_adapted:  []
No shortest paths disrupted for flood_DERP_RW_H_4326_2080411370. No economic impact.

-- CALCULATIONS FOR flood_DERP_RW_H_4326_2080416210 --
disrupted_edges baseline:  [(166, 2916), (203, 1668), (461, 463), (462, 532), (463, 461), (532, 462), (598, 2727), (717, 1737), (1392, 1940), (1394, 2728), (1398, 1937), (1433, 1438), (1437, 2650), (1438, 1433), (1654, 1669), (1668, 203), (1669, 1654), (1733, 1738), (1734, 1736), (1735, 1738), (1736, 1734), (1736, 1737), (1737, 1736), (1737, 717), (1738, 1735), (1738, 1733), (1937, 1398), (1940, 1392), (2650, 1437), (2727, 598), (2728, 1394), (2871, 2913), (2913, 2916), (29

In [16]:
for hazard_map, impact in event_impacts.items():
    if hazard_map in event_impacts_adapted.keys():
        print(f'\nEconomic impact (adapted) for {hazard_map}: {event_impacts_adapted[hazard_map]:.2f} EUR')
        print(f'Economic impact (baseline) for {hazard_map}: {impact:.2f} EUR')
        change=(1-(event_impacts_adapted[hazard_map]/impact))*100
        print(f'Reduction: ', change, '%')


Economic impact (adapted) for flood_DERP_RW_H_4326_2080411370: 0.00 EUR
Economic impact (baseline) for flood_DERP_RW_H_4326_2080411370: 29480151.25 EUR
Reduction:  100.0 %

Economic impact (adapted) for flood_DERP_RW_H_4326_2080416210: 0.00 EUR
Economic impact (baseline) for flood_DERP_RW_H_4326_2080416210: 37615464.08 EUR
Reduction:  100.0 %

Economic impact (adapted) for flood_DERP_RW_H_4326_2080418880: 0.00 EUR
Economic impact (baseline) for flood_DERP_RW_H_4326_2080418880: 29453063.97 EUR
Reduction:  100.0 %

Economic impact (adapted) for flood_DERP_RW_L_4326_2080411370: 29480151.25 EUR
Economic impact (baseline) for flood_DERP_RW_L_4326_2080411370: 29480151.25 EUR
Reduction:  0.0 %

Economic impact (adapted) for flood_DERP_RW_L_4326_2080416210: 0.00 EUR
Economic impact (baseline) for flood_DERP_RW_L_4326_2080416210: 37615464.08 EUR
Reduction:  100.0 %

Economic impact (adapted) for flood_DERP_RW_L_4326_2080418880: 0.00 EUR
Economic impact (baseline) for flood_DERP_RW_L_4326_20804

In [17]:
# EXPAND FOR VISUALISATION SCRIPT
# Create visualisation for the basin and the discharge points
from lonboard import Map, PolygonLayer, PathLayer, BaseLayer
import ast
# MIRACA color scheme
color_string = config.get('DEFAULT', 'miraca_colors')
miraca_colors = ast.literal_eval(color_string)

basin_id = 2080411370

    
# Set path for basin to add to visualization
basin_path = rf'C:\Data\Floods\basins\hybas_eu_{int(basin_id)}.shp'

# Generate the basin layer
basin = gpd.read_file(basin_path)
layer_basin = PolygonLayer.from_geopandas(basin,
    get_fill_color=miraca_colors['grey_200'],
    get_line_color=miraca_colors['primary blue'], get_line_width=70,
    auto_highlight=False,
    filled=True, opacity=0.2)

# Set path for the protected area to add to visualization
# adapt_path = rf'C:\Data\interim\adaptations\test_haz_level_adapt.geojson'
adapt_area = gpd.read_file(adapt_path)
layer_protected_area = PolygonLayer.from_geopandas(adapt_area,
    get_fill_color=miraca_colors['green_success'],
    get_line_color=miraca_colors['green_400'], get_line_width=30,
    auto_highlight=False,
    filled=True, 
    opacity=0.2)

# Create layer for assets for visualization
layer_assets = PathLayer.from_geopandas(assets.drop(columns=['buffered']), get_width=2, get_color=miraca_colors['black'], auto_highlight=True, )
affected_assets = [asset_id for asset_id in list(set(overlay_assets.asset.values))]
layer_affected_assets = PathLayer.from_geopandas(assets.iloc[affected_assets].drop(columns=['buffered']), get_width=3, get_color=miraca_colors['red_danger'], auto_highlight=True)
layer_protected_assets = PathLayer.from_geopandas(adapted_assets.drop(columns=['buffered']), get_width=4, get_color=miraca_colors['green_success'], auto_highlight=True)
layers_assets = [layer_assets, layer_affected_assets, layer_protected_assets]
# Flood return period: H for frequent(RP10-20), M for 100 year return period (RP100) and L for extreme (RP2000)
return_period_str='M'
# Generate flood layers and protection layers for visualization
flood_plot_path=rf'Floods\Germany\basin_intersections\DERP_RW_{return_period_str}_4326_hybas_intersections\flood_DERP_RW_{return_period_str}_4326_{basin_id}.geojson'
flood_m = data_path / flood_plot_path
flood_gdf=gpd.read_file(flood_m)
layers_flood=[]
f_area_colors = {1:'blue', 3:'green'}
for f_area in flood_gdf.flood_area.unique():
    for f_depth in flood_gdf.depth_class.unique():
        subset_gdf = flood_gdf[(flood_gdf.depth_class==f_depth) & (flood_gdf.flood_area==f_area)]
        if not subset_gdf.empty:
            color_key=f'{f_area_colors[f_area]}_{f_depth}00'
            layers_flood.append(PolygonLayer.from_geopandas(subset_gdf, 
                                                            get_fill_color=miraca_colors[color_key], 
                                                            opacity=0.5, 
                                                            stroked=False))


layers=[]
if layer_basin is not None:
    layers.append(layer_basin)
else:
    print('No basin layer')

if layer_protected_area is not None:
    layers.append(layer_protected_area)
else:
    print('No protected area layer')

if layers_flood is not None:
    layers.extend(layers_flood)
else:
    print('No flood layers')

if layers_assets is not None:
    layers.extend(layers_assets)
else:
    print('No asset layer')
Voyager = 'https://basemaps.cartocdn.com/gl/voyager-gl-style/style.json'


# m=Map(layer_affected_assets, show_tooltip=True, basemap_style=Voyager)
m = Map(layers, show_tooltip=True, basemap_style=Voyager)



c:\Users\peregrin\AppData\Local\anaconda3\envs\ci_adapt_env\Lib\site-packages\lonboard\_geoarrow\ops\reproject.py:97: UserWarning: Input being reprojected to EPSG:4326 CRS
  warnings.warn("Input being reprojected to EPSG:4326 CRS")
c:\Users\peregrin\AppData\Local\anaconda3\envs\ci_adapt_env\Lib\site-packages\lonboard\_geoarrow\ops\reproject.py:97: UserWarning: Input being reprojected to EPSG:4326 CRS
  warnings.warn("Input being reprojected to EPSG:4326 CRS")
c:\Users\peregrin\AppData\Local\anaconda3\envs\ci_adapt_env\Lib\site-packages\lonboard\_geoarrow\ops\reproject.py:97: UserWarning: Input being reprojected to EPSG:4326 CRS
  warnings.warn("Input being reprojected to EPSG:4326 CRS")


In [18]:

m



Map(basemap_style='https://basemaps.cartocdn.com/gl/voyager-gl-style/style.json', layers=[PolygonLayer(filled=…